In [11]:
import time 
import sys
from blu import *  
# True parameters
data=np.load(data_dir+'/catalog/m1m2zxeff_posterior_138_lensed.npz')
m1,m2,redshift,xeff = data['m1'],data['m2'],data['redshift'], data['xeff'] 
del data
# posterior samples 
with open('./posterior/img1_det_m1m2xeff.pkl', 'rb') as f:
    img1=dill.load(f)
with open('./posterior/img2_det_m1m2xeff.pkl', 'rb') as f:
    img2=dill.load(f)
# uniform model
with open('./pop_prior/uniform_pop_m1m2xeff.pkl', 'rb') as f:
    pop_uni=dill.load(f)
# HDPGMM models 
with open('./pop_prior/astro_m1m2chieff.pkl', 'rb') as f:
    pop_model=dill.load(f)
# benchmark model
pop_true = mix_pop('./pop_prior/real_dist.pkl')
# power-law model (wrong)
pop_power = mix_pop('./pop_prior/PL_pdf.pkl',wide=True)
N = 1e4
OLU = OddsRatio(gw_pop=pop_model,Nmc=N)
OLU_true = OddsRatio(gw_pop=pop_true,Nmc=N)
OLU_pl = OddsRatio(gw_pop=pop_power,Nmc=N)
OLU_uni = OddsRatio(gw_pop=pop_uni,Nmc=N)

In [21]:
# only compute the blu within the reliable region of reconstructed astro population function
m1z = m1z=m1*(1+redshift)
m1z = m1z[:int(m1z.size/2)]
index = np.where(m1z<90)[0]

In [ ]:
blu_uni=[]
blu_true=[]
blu_hdpgmm=[]
blu_pl=[]
#blu_obs=[]
t0 = time.time()
for i in tqdm(index, total = int(index.size), desc = '-th pair'):
    blu_true.append(OLU_true.BayesFactor_PEuniform(img1[i],img2[i]))
    blu_hdpgmm.append(OLU.BayesFactor_PEuniform(img1[i],img2[i]))
    blu_pl.append(OLU_pl.BayesFactor_PEuniform(img1[i],img2[i]))
    blu_uni.append(OLU_uni.BayesFactor_PEuniform(img1[i],img2[i]))
    if i == index[0]: print(time.time()-t0)
    
blu_true = np.array(blu_true)
blu_hdpgmm = np.array(blu_hdpgmm)
blu_pl = np.array(blu_pl)
blu_uni = np.array(blu_uni)

blu_true = log_blu(blu_true)
blu_hdpgmm = log_blu(blu_hdpgmm)
blu_pl = log_blu(blu_pl)
blu_uni = log_blu(blu_uni)


11.284301996231079
10.703462839126587
10.622402667999268
11.037400007247925
10.273090124130249
10.165321111679077
11.0977783203125
10.200114727020264
10.403053045272827
10.530593156814575
10.315058946609497
9.721719980239868
10.456326007843018
9.981173038482666
9.917980909347534
10.744271755218506
10.135848999023438
10.387354850769043
10.593093872070312
9.980990886688232
10.58449387550354
10.824148893356323
10.26425290107727
11.04597806930542


In [ ]:
from marginal_plt import *
%matplotlib inline

xlim=(18,120)
ylim=(-2.1,4.3)
n = blu_true.shape[0]
plotting_two(blu_true[:n], blu_uni[:n],pop_model,
             pop_true,pop_power, m1z[:n],math_blu,
             real_dist=True,uni_dist=True,xlim=xlim,ylim=ylim)
plt.savefig('blu_uni.pdf',bbox_inches = 'tight')
#ylim=(0,3.1)
plotting_two(blu_true[:n], blu_pl[:n],pop_model,
             pop_true,pop_power, m1z[:n],math_blu,
             real_dist=True,PL_dist=True,xlim=xlim,ylim=ylim)
plt.savefig('blu_pl.pdf',bbox_inches = 'tight')
#ylim=(0.5,2.26)
#ylim=(0,5.1)
plotting_two(blu_true[:n], blu_hdpgmm[:n],pop_model,
             pop_true,pop_true, m1z[:n],math_blu,
             real_dist=True,HDPGMM_model=True,xlim=xlim,ylim=ylim)
plt.savefig('blu_dpgmm.pdf',bbox_inches = 'tight')